In [311]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver import ActionChains as AC
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [350]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
driver.implicitly_wait(10)

In [351]:
driver.get('https://basketball.realgm.com/international/league/4/Spanish-ACB/team/649/Acunsa-GBC/stats/2021/Per_Minute/All/All/points/All/desc/1/Regular_Season')
driver.find_element_by_class_name('Frame-sc-1d4hofp-0.czVBsw').click()

In [31]:
#driver.delete_all_cookies()

In [91]:
uno = driver.find_elements_by_class_name('has-subnav.with-js')
dos = uno[5].click()

In [92]:
driver.find_element_by_class_name('Frame-sc-1d4hofp-0.czVBsw').click()
driver.find_element_by_xpath('//*[@id="site-takeover"]/div[3]/div/div[1]/div[5]/select').click()
driver.find_element_by_xpath('//*[@id="site-takeover"]/div[3]/div/div[1]/div[5]/select/option[6]').click()
driver.find_element_by_class_name('Frame-sc-1d4hofp-0.czVBsw').click()


### With this cell we get the name of each stat column.

In [301]:
tablita = driver.find_element_by_class_name("tablesaw.compact.tablesaw-swipe")
columnas = tablita.find_elements_by_tag_name("th")
interm = []
for a in columnas:
    interm.append(a)
columns_name = []
for name in interm: 
    columns_name.append(name.text)

### With this cell we get all the stats/minute of every player in this season (2020-21).

In [140]:
busqueda = driver.find_element_by_id("teamnav")
equipos = busqueda.find_elements_by_class_name("has-subnav")


In [141]:
len(equipos)

19

In [354]:
buscador = driver.find_element_by_xpath('//*[@id="site-takeover"]/div[3]/div/div[1]/div[4]/select')
options = buscador.find_elements_by_tag_name("option") #get all the options into a list

optionsList = []
for option in options: #iterate over the options, place attribute value in list
    optionsList.append(option.get_attribute("value"))
optionsList = optionsList[1:]

stats_list = [] 
final = []
for optionValue in optionsList: 
    print("starting loop on option %s" % optionValue)
    select = Select(driver.find_element_by_xpath('//*[@id="site-takeover"]/div[3]/div/div[1]/div[4]/select'))
    select.select_by_value(optionValue)
    tablita = driver.find_element_by_class_name("tablesaw.compact.tablesaw-swipe")
    players = tablita.find_elements_by_tag_name("tr")
    for player in players:
        stats = player.find_elements_by_tag_name("td")
        for stat in stats:
            stats_list.append(stat.text)

starting loop on option /international/league/4/Spanish-ACB/team/649/Acunsa-GBC/stats/2021/Per_Minute/All/All/points/All/desc/1/Regular_Season
starting loop on option /international/league/4/Spanish-ACB/team/23/Barca/stats/2021/Per_Minute/All/All/points/All/desc/1/Regular_Season
starting loop on option /international/league/4/Spanish-ACB/team/473/BAXI-Manresa/stats/2021/Per_Minute/All/All/points/All/desc/1/Regular_Season
starting loop on option /international/league/4/Spanish-ACB/team/129/Casademont-Zaragoza/stats/2021/Per_Minute/All/All/points/All/desc/1/Regular_Season
starting loop on option /international/league/4/Spanish-ACB/team/16/Club-Joventut-Badalona/stats/2021/Per_Minute/All/All/points/All/desc/1/Regular_Season
starting loop on option /international/league/4/Spanish-ACB/team/212/Coosur-Real-Betis/stats/2021/Per_Minute/All/All/points/All/desc/1/Regular_Season
starting loop on option /international/league/4/Spanish-ACB/team/33/Herbalife-Gran-Canaria/stats/2021/Per_Minute/All/Al

In [365]:
len(stats_list)

7866

In [369]:
final = np.array_split(stats_list, 342)

In [371]:
acb_dataset_per_minute = pd.DataFrame(final, columns = columns_name)

In [373]:
acb_dataset_per_minute

,#,Player,Team,GP,MIN,FGM,FGA,FG%,3PM,3PA,...,FT%,TOV,PF,ORB,DRB,REB,AST,STL,BLK,PTS
0,1,Jaime Echenique,GBC,16,22.1,0.2,0.4,.544,0.0,0.0,...,.797,0.1,0.1,0.1,0.1,0.2,0.0,0.1,0.0,0.6
1,2,Johnny Dee,GBC,33,20.2,0.2,0.4,.396,0.1,0.3,...,.846,0.1,0.1,0.0,0.1,0.1,0.1,0.0,0.0,0.5
2,3,Viny Okouo,GBC,33,19.0,0.2,0.4,.512,0.0,0.0,...,.604,0.1,0.1,0.1,0.2,0.3,0.0,0.0,0.0,0.5
3,4,Dino Radoncic,GBC,33,27.8,0.1,0.3,.446,0.0,0.1,...,.755,0.1,0.1,0.1,0.1,0.2,0.1,0.0,0.0,0.4
4,5,Mike Carlson,GBC,16,16.1,0.1,0.3,.442,0.1,0.2,...,.538,0.0,0.1,0.0,0.1,0.1,0.0,0.0,0.0,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,12,Jaime Pradilla,VAL,25,8.7,0.1,0.3,.554,0.0,0.0,...,.643,0.1,0.1,0.1,0.1,0.2,0.0,0.0,0.0,0.3
338,13,Guillem Ferrando,VAL,2,3.0,0.0,0.3,.000,0.0,0.2,...,1.000,0.3,0.2,0.0,0.2,0.2,0.2,0.0,0.0,0.3
339,14,Josep Puerto,VAL,16,8.6,0.1,0.2,.462,0.1,0.1,...,.778,0.0,0.1,0.0,0.1,0.1,0.0,0.0,0.0,0.3
340,15,Guillem Vives,VAL,25,19.5,0.1,0.2,.385,0.0,0.1,...,.769,0.1,0.1,0.0,0.1,0.1,0.1,0.1,0.0,0.2


In [374]:
acb_dataset_per_minute.to_csv("stats_per_minute.csv")

### Scraping ACB Tables with averages stats

In [224]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
driver.implicitly_wait(10)
driver.get('http://acb.com/club/estadisticas/id/7/temporada_id/2020/edicion_id/undefined')

In [232]:
primera = driver.find_element_by_tag_name("thead")
segunda = primera.find_elements_by_tag_name("tr")
headers = []
for r in segunda:
    td = r.find_elements_by_tag_name("th")
    for d in td:
        headers.append(d.text)

In [233]:
headers = headers[17:]

In [235]:
headers = ['NADA','PLAYER_NAME', 'GP','MIN/G','5i','PPG','3FGM','3FGA','3FG%','2FGM','2FGA','2FG%','FTM','FTA','FT%',
           'DREB','OREB','TREB','ASIST','ST','TO','BLK','RBLK','DUNKS','PF','RPF','+/-','VAL']

### We use Beautiful Soup to get the stats tables for each player.

In [360]:
url = "http://acb.com/club/estadisticas/id/"
acb_media = []
for team in range(0,20):
    html = requests.get(url+"{}".format(team)+"/temporada_id/2020/edicion_id/undefined")
    res = BeautifulSoup(html.content,"html.parser")
    #Aquí faltaría un range de los jugadores
    table = res.find("tbody")
    rows = table.findAll("tr")
    for r in rows:
        elements = r.findAll("td")
        jugador = [e.getText().strip() for e in elements]
        acb_media.append(jugador)

309

In [362]:
acb_dataset = pd.DataFrame(acb_media, columns = headers)
index = acb_dataset[acb_dataset["PLAYER_NAME"] == "Totales"].index
acb_dataset.drop(index, inplace=True)
acb_dataset = acb_dataset.drop(["NADA", "5i"], axis = 1)

In [363]:
acb_dataset

,PLAYER_NAME,GP,MIN/G,PPG,3FGM,3FGA,3FG%,2FGM,2FGA,2FG%,...,ASIST,ST,TO,BLK,RBLK,DUNKS,PF,RPF,+/-,VAL
2,Nikola Mirotic,28,20:52,"14,8","1,8","3,7","48,5%","3,4","4,9","68,6%",...,"0,8","0,9","1,6","0,5","0,2","0,1","1,5","3,1","9,0","17,8"
3,Cory Higgins,23,21:16,"10,1","0,7","1,9","39,5%","2,7","5,3","50,0%",...,"2,4","1,3","1,0","0,3","0,4","0,3","1,6","2,9","6,0","12,1"
4,Pau Gasol,6,15:05,"8,8","0,2","0,5","33,3%","3,3","6,3","52,6%",...,"1,3","0,2","1,0","0,7","0,2","0,2","1,2","2,7","4,0","11,8"
5,Brandon Davies,25,16:47,"9,9","0,0","0,2","25,0%","3,4","6,0","56,3%",...,"1,3","0,6","1,5","0,4","0,5","0,4","2,4","3,6","6,0","11,8"
6,Nick Calathes,32,19:29,"7,8","0,8","2,1","40,3%","2,3","4,6","50,3%",...,"5,4","1,1","1,8","0,1","0,3","0,0","2,3","1,3","8,0","10,3"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,Rodijs Macoha,1,0:39,"0,0","0,0","0,0","0,0%","0,0","0,0","0,0%",...,"0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0"
302,Tomás Bellas,3,13:34,"0,7","0,0","1,0","0,0%","0,3","2,0","16,7%",...,"1,3","0,3","1,3","0,0","0,3","0,0","1,0","0,7","-2,0","-1,7"
303,Osas Ehigiator,7,7:48,"0,0","0,0","0,0","0,0%","0,0","0,6","0,0%",...,"0,1","0,3","1,0","0,3","0,0","0,0","1,9","0,4","-3,0","-0,9"
304,Njegos Sikiras,2,2:19,"0,0","0,0","0,0","0,0%","0,0","0,0","0,0%",...,"0,5","0,0","0,5","0,0","0,0","0,0","1,0","0,0","0,0","-0,5"


In [375]:
acb_dataset.to_csv("acb_stats.csv")